In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pickle
from steenroder import *

In [2]:
# with open('notebooks/cone_rp4.pkl', 'rb') as f:
#     filtration = pickle.load(f)

In [3]:
filtration = (
    (0,), 
    (1,), (0,1), 
    (2,), (0,2), (1,2), (0,1,2),
    (3,), (0,3), (1,3), (0,1,3), (2,3),
    (4,), (0,4), (1,4), (2,4), (1,2,4), (3,4), (0,3,4), (2,3,4),
    (5,), (0,5), (1,5), (2,5), (0,2,5), (3,5), (1,3,5), (2,3,5), (4,5), (0,4,5), (1,4,5)
    )

# New approach

In [4]:
filtration_by_dim, spx_filtration_idx_by_dim = sort_filtration_by_dim(filtration, maxdim=None)
matrix_gen = gen_coboundary_by_dim(filtration_by_dim, spx_filtration_idx_by_dim)
result_after_reduction = get_reduced_triangular_sparse(matrix_gen)
result_after_reduction

[(array([20, 12,  7,  3,  1,  0]),
  ([[21, 22, 23, 25, 28],
    [13, 14, 15, 17, 28],
    [8, 9, 11, 17, 25],
    [4, 5, 11, 15, 23],
    [2, 5, 9, 14, 22],
    []],
   [[20], [12], [7], [3], [1], [0, 1, 3, 7, 12, 20]])),
 (array([28, 25, 23, 22, 21, 17, 15, 14, 13, 11,  9,  8,  5,  4,  2]),
  ([[29, 30],
    [26, 27],
    [24, 27],
    [27, 30],
    [],
    [18, 19],
    [16, 19],
    [19, 30],
    [],
    [],
    [10, 26],
    [],
    [6, 16],
    [],
    []],
   [[28],
    [25],
    [23],
    [22, 25],
    [21, 22, 23, 25, 28],
    [17],
    [15],
    [14, 15],
    [13, 14, 15, 17, 28],
    [11, 14, 15, 22, 25],
    [9],
    [8, 9, 14, 15, 17, 22],
    [5],
    [4, 5, 14, 22, 23, 25],
    [2, 5, 9, 14, 22]])),
 (array([30, 29, 27, 26, 24, 19, 18, 16, 10,  6]),
  ([[], [], [], [], [], [], [], [], [], []],
   [[30], [29], [27], [26], [24], [19], [18], [16], [10], [6]]))]

In [5]:
barcode = get_barcode_from_sparse(filtration, result_after_reduction)
barcode

[[(30, inf)],
 [(9, 10), (17, 18), (19, inf), (22, 23), (26, 27), (28, 29)],
 [(0, inf),
  (1, 2),
  (3, 8),
  (4, 5),
  (6, 7),
  (11, 16),
  (12, 13),
  (14, 15),
  (20, 21),
  (24, 25)]]

In [16]:
[filtration[30 - x] for x in coho_reps[0][0]]

[(0,), (1,), (2,), (3,), (4,), (5,)]

In [6]:
coho_reps = get_coho_reps_from_sparse(filtration, barcode, result_after_reduction)
coho_reps

[[[30, 29, 27, 23, 18, 10]],
 [[9, 8, 7, 5, 2],
  [17, 16, 15, 13, 2],
  [19, 16, 15, 8, 5],
  [22, 21, 19, 13, 5],
  [26, 25, 19, 15, 7],
  [28, 25, 21, 16, 8]],
 [[0],
  [1, 0],
  [3, 0],
  [4, 3],
  [6, 3],
  [11, 0],
  [12, 11],
  [14, 11],
  [20, 4],
  [24, 14]]]

In [18]:
k = 1
get_steenrod_matrix_sparse(k, coho_reps, filtration, spx_filtration_idx_by_dim)

[[[]], [[]], [[], [0, 1], [3], [12, 11, 3, 4], [14, 6], [24, 20]]]

# Using the previous approach

In [8]:
matrix = get_coboundary(filtration)
reduced, triangular = get_reduced_triangular(matrix)

In [9]:
barcode = get_barcode(filtration, reduced)
barcode

[(0, inf),
 (1, 2),
 (3, 8),
 (4, 5),
 (6, 7),
 (9, 10),
 (11, 16),
 (12, 13),
 (14, 15),
 (17, 18),
 (19, inf),
 (20, 21),
 (22, 23),
 (24, 25),
 (26, 27),
 (28, 29),
 (30, inf)]

In [10]:
filter_barcode_by_dim(barcode, filtration)

{0: [(30, inf)],
 1: [(9, 10), (17, 18), (19, inf), (22, 23), (26, 27), (28, 29)],
 2: [(0, inf),
  (1, 2),
  (3, 8),
  (4, 5),
  (6, 7),
  (11, 16),
  (12, 13),
  (14, 15),
  (20, 21),
  (24, 25)]}

In [11]:
cohoreps = get_coho_reps(filtration, barcode, reduced, triangular)
for j in range(len(barcode)):
    print(np.flatnonzero(cohoreps[:, j]))

[0]
[0 1]
[0 3]
[3 4]
[3 6]
[2 5 7 8 9]
[ 0 11]
[11 12]
[11 14]
[ 2 13 15 16 17]
[ 5  8 15 16 19]
[ 4 20]
[ 5 13 19 21 22]
[14 24]
[ 7 15 19 25 26]
[ 8 16 21 25 28]
[10 18 23 27 29 30]


In [12]:
steenrod_matrix = get_steenrod_matrix(k, cohoreps, barcode, filtration)

# Check that the reduced and triangular matrices are the same in both approaches

In [10]:
N = len(filtration)
for idxs, (red, tri) in result_after_reduction:
    for i in range(len(idxs)):
        assert set(N - 1 - np.flatnonzero(reduced[:, N - 1 - idxs[i]])) == red[i]
        assert set(N - 1 - np.flatnonzero(triangular[:, N - 1 - idxs[i]])) == tri[i]

In [ ]:
# N = len(filtration)
# matrix_gen = gen_coboundary_by_dim(filtration)
# for idxs, data in matrix_gen:
#     print(idxs)
#     for i in range(len(idxs)):
#         assert set(N - 1 - np.flatnonzero(matrix[:, N - 1 - idxs[i]])) == set(data[i])